# TASK 3

Создайте соединение с базой данных с помощью библиотеки sqlite3.

In [31]:
import pandas as pd
import sqlite3 as sq
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
conn = sq.connect('/content/drive/MyDrive/Google_Drive/School21/Day11/checking-logs.sqlite')

Создайте новую таблицу datamart в базе данных, объединив таблицы pageviews и checker с помощью только одного запроса.
Таблица должна содержать следующие столбцы: uid, labname, first_commit_ts, first_view_ts.
first_commit_ts — это просто новое имя для столбца timestamp из таблицы checker; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
first_view_ts — первое посещение пользователем из таблицы pageviews, метка времени посещения пользователем ленты новостей.
По-прежнему нужно использовать фильтр status = 'ready'.
По-прежнему нужно использовать фильтр numTrials = 1.
Имена лабораторных заданий по-прежнему должны быть из следующего списка: laba04, laba04s, laba05, laba06, laba06s, project1.
Таблица должна содержать только пользователей (uid с user_*), а не администраторов.
first_commit_ts и first_view_ts должны быть распарсены как datetime64[ns]

In [4]:
pd.io.sql.read_sql('PRAGMA table_info(checker);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [5]:
pd.io.sql.read_sql('PRAGMA table_info(pageviews);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


In [6]:
datamart_1 = pd.read_sql("""SELECT uid, labname, timestamp as first_commit_ts  
                        FROM checker 
                        WHERE uid like 'user%' AND status like 'ready' AND numTrials == 1 AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')""", conn)

In [7]:
datamart_1

,uid,labname,first_commit_ts
0,user_4,project1,2020-04-17 05:19:02.744528
1,user_4,laba04,2020-04-17 11:33:17.366400
2,user_4,laba04s,2020-04-17 11:48:41.992466
3,user_17,project1,2020-04-18 07:56:45.408648
4,user_30,laba04,2020-04-18 13:36:53.971502
...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205
136,user_19,laba06s,2020-05-21 13:27:06.705881
137,user_23,laba06s,2020-05-21 14:29:15.709568
138,user_17,laba06,2020-05-21 15:21:31.567615


In [8]:
datamart_2 = pd.read_sql("""SELECT uid, min(datetime) as first_view_ts 
                        FROM pageviews
                        GROUP BY uid
                         """, conn)

In [9]:
datamart_2

,uid,first_view_ts
0,admin_0,2020-04-22 10:45:25.601360
1,admin_1,2020-04-17 12:01:08.463179
2,admin_2,2020-04-28 19:09:01.362373
3,admin_3,2020-04-17 12:17:39.287778
4,user_1,2020-04-26 21:53:59.624136
5,user_10,2020-04-18 12:19:50.182714
6,user_14,2020-04-18 10:53:52.623447
7,user_17,2020-04-18 10:56:55.833899
8,user_18,2020-04-26 22:49:29.243278
9,user_19,2020-04-21 20:30:38.034966


In [17]:
datamart = pd.read_sql("""SELECT table_1.uid, labname, first_commit_ts, first_view_ts  
                        FROM 
                        (SELECT uid, labname, timestamp as first_commit_ts
                        FROM checker
                        WHERE uid like 'user%' AND status like 'ready' AND numTrials == 1 AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')) as table_1
                        LEFT JOIN 
                        (SELECT uid, min(datetime) as first_view_ts 
                        FROM pageviews
                        GROUP BY uid) as table_2 on table_1.uid = table_2.uid
                         """, conn, parse_dates=['first_commit_ts', 'first_view_ts'])

In [18]:
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


Используя методы библиотеки Pandas, создайте два датафрейма: test и control.
test должен включать пользователей, у которых имеются значения в first_view_ts.
control должен включать пользователей, у которых отсутствуют значения в first_view_ts.
Замените пропущенные значения в control средним значением first_view_ts пользователей из test (оно пригодится нам для анализа в будущем).
Сохраните обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).

In [25]:
datamart.to_sql('datamart', con = conn, if_exists='replace')

In [26]:
pd.read_sql('PRAGMA table_info(datamart)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [27]:
test = datamart[pd.notnull(datamart['first_view_ts'])]

In [ ]:
test

In [29]:
control = datamart[pd.isnull(datamart['first_view_ts'])]

In [30]:
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
5,user_2,laba04,2020-04-18 13:42:35.482008,NaT
6,user_2,laba04s,2020-04-18 13:51:22.291271,NaT
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,NaT
132,user_6,laba06s,2020-05-20 14:50:07.609937,NaT
134,user_7,laba06s,2020-05-20 23:05:37.742597,NaT
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT


In [33]:
control['first_view_ts'] = test['first_view_ts'].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783552
132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783552
134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783552
135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783552


In [35]:
test.to_sql('test', con = conn, if_exists='replace')

In [36]:
control.to_sql('control', con = conn, if_exists='replace')

Закройте соединение с базой данных.

In [38]:
conn.close()